# Welcome to the FunC interactive tutorial!
Language basics will be covered, more information can be found in the TON repo:
1. [Tests](https://github.com/ton-blockchain/ton/tree/9c9248a9ae1791152d88c9c36574b4becdf5e8b6/crypto/func/test)
2. [Sample contracts](https://github.com/ton-blockchain/ton/tree/9c9248a9ae1791152d88c9c36574b4becdf5e8b6/crypto/smartcont)
3. [TVM docs](https://test.ton.org/tvm.pdf)
4. [Fift docs](https://test.ton.org/fiftbase.pdf)

## Xeus-Fift specific features

In [57]:
#include "stdlib.fc"  ;; 

Xeus-fift error: failed to load `stdlib.fc`

## Integer operations
* Arithmetic `+`  `-`  `*`  `/`  `%`   
* Bitwise `&`  `|`  `<<`  `>>`   
* Boolean `<`  `<=`  `>`  `>=`  `==`  `!=` 

In [35]:
0x01 ^ 0x80 ;; xor

129

In [54]:
8 <=> -4 ;; compare (1 if equal, -1 if less, 0 otherwise)

1

In [37]:
~ 0xff + 0xff ;; bitwise complement

-1

In [38]:
9 ~>> 2 ;; division by 2^n using nearest-integer rounding

2

In [39]:
9 ^>> 2 ;; division by 2^n using ceiling rounding

3

Rounding works similarly for integer division and modular division operators:  
`~/`  `^/`  `~%`  `^%`

In [40]:
11 /% 2 ;; extended mod division: computes both the quotient and the remainder

5 1

In [65]:
int x = 42;
x += 2

44

In [66]:
true

-1

In [67]:
false

0

## Tuple manipulation

In [41]:
var t = (1, 2, 3);  ;; packing
t

1 2 3

In [42]:
var (a, b, c) = (1, 2, 3); ;; unpacking
b

2

In [59]:
var tr = triple(1, 2, 3);

[ 1 2 3 ]

In [63]:
var tr = triple(1, 2, 3);
first(tr) ;; also available `first`, `third`, `fourth`, or you can define your helper

1

In [69]:
untriple(triple(1, 2, 3))

1 2 3